# XCALE Master thesis : DBN Building v2

In this notebook, we build the dbn dataset

<style>.table {margin-left:0px}</style>

## Import

In [735]:
# data handling
import numpy as np
import pandas as pd
from pylab import rcParams
rcParams['figure.figsize'] = 18, 10

In [736]:
# Database
#!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install mysql-connector-python
from mysql.connector import connect
# error
import traceback
from collections import defaultdict
from datetime import time, timedelta, datetime
# utilitaries
import functools
import tqdm

## Database

In [737]:
# Log
_db_config = {
    'user': 'cajuge',
    'password': 'Kz1773qMWIVhRZUZ',
    'host': 'franceioi.cinniket56wn.eu-central-1.rds.amazonaws.com',
    'database': 'srl',
    'port':'3306'
}

In [738]:
_tables = ["clavier", "modification",
          "navigation", "pas_a_pas","srl_final_prompt",
          "srl_initial_prompt","srl_prompt","validation", "connexion"]

In [739]:
def get_connection(config):
    return connect(**config)
def close_connection(connection):
    connection.close()

In [740]:
def getTablesFromDB(min_id_connection, _tables):
    dataframe_tables = {}
    mysql_connection = get_connection(_db_config)
    for table in _tables:
        with mysql_connection.cursor() as cursor:
            try:
                # ----------- Get every connection / participant ---------- #
                if table == "connexion":
                  query_select = "SELECT c.id, c.timestamp FROM connexion c INNER JOIN participer p ON p.id = c.id_participation WHERE c.id >= %s AND p.id_activite=11 ORDER BY c.id, c.timestamp"
                  cursor.execute(query_select, (min_id_connection,))
                elif table == "validation":
                  query_select = "SELECT t.experimentation, t.score, t.id_connexion, t.timestamp FROM {} t INNER JOIN connexion c ON c.id = t.id_connexion INNER JOIN participer p ON p.id = c.id_participation WHERE c.id >= %s AND p.id_activite=11 ORDER BY c.id, c.timestamp".format(table)
                  cursor.execute(query_select, (min_id_connection,))
                else:
                  query_select = "SELECT t.id_connexion, t.timestamp FROM {} t INNER JOIN connexion c ON c.id = t.id_connexion INNER JOIN participer p ON p.id = c.id_participation WHERE c.id >= %s AND p.id_activite=11 ORDER BY c.id, c.timestamp".format(table)
                  cursor.execute(query_select, (min_id_connection,))
                
                result = cursor.fetchall()
                columns_name = np.array(
                    [cursor.description[i][0] for i in range(len(cursor.description)) if cursor.description[i][0]])
                data = []
                for row in result:
                    data.append(np.array([row[i] for i in range(len(row))]))
                dataframe_tables[table] = pd.DataFrame(data, columns=columns_name)
            except Exception:
                traceback.print_exc()
    
    
    return dataframe_tables

def getTablesFromDB_ids(ids, _tables):
    dataframe_tables = {}
    mysql_connection = get_connection(_db_config)
    for table in _tables:
        with mysql_connection.cursor() as cursor:
            try:
                # ----------- Get every connection / participant ---------- #
                format_strings = ','.join(['%s'] * len(ids))
                if table == "connexion":
                  query_select = "SELECT * FROM connexion WHERE id IN ({}) ORDER BY id, timestamp".format(format_strings)
                else:
                  query_select = "SELECT * FROM {} WHERE id_connexion IN ({}) ORDER BY id_connexion, timestamp".format(table, format_strings)
                
                cursor.execute(query_select, tuple(ids))
                result = cursor.fetchall()
                columns_name = np.array(
                    [cursor.description[i][0] for i in range(len(cursor.description)) if cursor.description[i][0]])
                data = []
                for row in result:
                    data.append(np.array([row[i] for i in range(len(row))]))
                dataframe_tables[table] = pd.DataFrame(data, columns=columns_name)
            except Exception:
                traceback.print_exc()
    
    
    return dataframe_tables

In [741]:
dataframe_tables = getTablesFromDB(191, _tables)

In [742]:
dataframe_tables["validation"].columns

Index(['experimentation', 'score', 'id_connexion', 'timestamp'], dtype='object')

In [743]:
dataframe_tables["connexion"].columns

Index(['id', 'timestamp'], dtype='object')

In [744]:
table = "connexion"
sujet_range = [19,20,21,22,23,24,25,26,27,28]
connections = dataframe_tables[table]
nb_connection_1h = []
for j, id_connexion in enumerate(connections.id):
  last_timestamp_table = []
  for table in [i for i in _tables if i != "connexion"]:
    sequence = dataframe_tables[table][dataframe_tables[table].id_connexion == id_connexion].timestamp
    if len(sequence) != 0:
      last_timestamp_table.append(max(sequence))
  if len(last_timestamp_table) != 0:
    if max(last_timestamp_table) - connections.timestamp[j] >= timedelta(hours=1):
      if dataframe_tables["validation"].query("id_connexion == {} and experimentation == 0 and score > 0".format(id_connexion)).shape[0] != 0:
        nb_connection_1h.append(id_connexion)
print("{} registered connections which last at least 1 hour".format(len(nb_connection_1h)))

89 registered connections which last at least 1 hour


In [745]:
dataframe_tables = getTablesFromDB_ids(nb_connection_1h, _tables)

## Building V.2

<img src='https://drive.google.com/uc?id=19I5DHRi0Z1gI4wwqRsFSIkEz1rh9w8x3' />


In [746]:
data = [[] for _ in nb_connection_1h]
columns = []

In [747]:
df = dataframe_tables.copy()

## Data Indicators

### Timestamps

In [748]:
def get_important_timestamps(ids_connexion, df_prompt_middle, df_prompt_final, df_connexion):
  timestamps_middle = []
  timestamps_final = []
  for id_connexion in ids_connexion:
    temp_df = df_prompt_middle.query('id_connexion == {}'.format(id_connexion))
    if temp_df.shape[0] == 0:
      # the user doesn't not provide us prompts
      timestamp = list(df_connexion.query('id == {}'.format(id_connexion)).timestamp)[0]
      timestamps_middle.append(timestamp + timedelta(minutes=30))
      timestamps_final.append(timestamp + timedelta(hours=1))
    else:
      timestamp = list(temp_df.timestamp)[0]
      timestamps_middle.append(timestamp)
      #final
      temp_df = df_prompt_final.query('id_connexion == {}'.format(id_connexion))
      if temp_df.shape[0] == 0:
        # the user doesn't not provide us prompts
        timestamps_final.append(timestamp + timedelta(minutes=30))
      else:
        timestamps_final(list(temp_df.timestamp)[0])
  return [i.to_pydatetime() for i in timestamps_middle], [i.to_pydatetime() for i in timestamps_final]

In [749]:
def get_index_timestamp(df, timestamp1, timestamp2):
  column = list(df.columns).index("timestamp")
  return [row for row in range(df.shape[0]) if df.iloc[row, column] > timestamp1 and df.iloc[row, column] <= timestamp2]

In [750]:
def draw(data, columns, nb_connection_1h):
  return pd.DataFrame(data[:5], columns=columns, index=nb_connection_1h[:5])

In [751]:
def eliminate_same_subject_version_score(df):
  indexes = []
  uniques = list(df[["id_sujet", "version"]].value_counts().index)
  for i in range(len(uniques)):
    df_multiple = df.query("id_sujet == {} and version == {}".format(uniques[i][0], uniques[i][1]))
    indexes.append(list(df_multiple.index)[-1])
  return indexes

In [752]:
def eliminate_same_subject_version_following(df, col1, col2):
  indexes = []
  memory = None
  for i in range(df.shape[0]):
    if memory == None:
      memory = (df.iloc[i,col1], df.iloc[i,col2])
      indexes.append(i)
    else:
      if df.iloc[i,col1] != memory[0] or df.iloc[i,col2] != memory[1]:
        memory = (df.iloc[i,col1], df.iloc[i,col2])
        indexes.append(i)
  return indexes

def eliminate_same_subject_version_other_following(df, col1, col2, col3):
  indexes = []
  memory = None
  for i in range(df.shape[0]):
    if memory == None:
      memory = (df.iloc[i,col1], df.iloc[i,col2], df.iloc[i,col3])
      indexes.append(i)
    else:
      if df.iloc[i,col1] != memory[0] or df.iloc[i,col2] != memory[1] or df.iloc[i,col3] != memory[2]:
        memory = (df.iloc[i,col1], df.iloc[i,col2], df.iloc[i,col3])
        indexes.append(i)
  return indexes

In [753]:
beginning_timestamp = [df["connexion"].query("id == {}".format(id)).iloc[0, 3] for id in nb_connection_1h]
middle_timestamp, end_timestamp = get_important_timestamps(nb_connection_1h, df["srl_prompt"].query("id_connexion in {}".format(nb_connection_1h)), 
                                                           df["srl_final_prompt"].query("id_connexion in {}".format(nb_connection_1h)), df["connexion"].query("id in {}".format(nb_connection_1h)))

### Rate of Time spent on validated subjects

Indicator name : 
* validated_time_rate

Computation : 
* Time spent on validated subject (help + subject) / 30 minutes

In [754]:
def validated_time_rate(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  column_val_sujet = list(df["validation"].columns).index("id_sujet")
  column_val_version = list(df["validation"].columns).index("version")
  column_nav_sujet = list(df["navigation"].columns).index("id_sujet")
  column_nav_version = list(df["navigation"].columns).index("version")
  column_nav_timestamp = list(df["navigation"].columns).index("timestamp")
  column_nav_module = list(df["navigation"].columns).index("module")
  for i,id in enumerate(nb_connection_1h):
    df_val = df["validation"].query("id_connexion == {} and score > 0 and experimentation == 0 and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    index = eliminate_same_subject_version_score(df_val)
    df_val = df_val.loc[index,df_val.columns]
    time_spent = timedelta(seconds=0)
    for j in range(df_val.shape[0]):
      sujet = df_val.iloc[j,column_val_sujet]
      version = df_val.iloc[j,column_val_version]
      df_nav = df["navigation"].query("id_connexion == {} and id_sujet in {}".format(id, sujet_range))
      index = get_index_timestamp(df_nav, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_nav, middle_timestamp[i], end_timestamp[i])
      df_nav = df_nav.iloc[index,:]
      index = eliminate_same_subject_version_following(df_nav, column_nav_sujet, column_nav_version)
      df_nav = df_nav.iloc[index,:]
      for k in range(df_nav.shape[0]):
        if df_nav.iloc[k,column_nav_sujet] == sujet and df_nav.iloc[k,column_nav_version] == version and df_nav.iloc[k,column_nav_module] == "Exercice":
          time = df_nav.iloc[k,column_nav_timestamp]
          for l in range(k+1, df_nav.shape[0]):
            if df_nav.iloc[l,column_nav_sujet] != sujet or df_nav.iloc[l,column_nav_version] != version:
              time = df_nav.iloc[l,column_nav_timestamp] - time
              break
          else:
              time = middle_timestamp[i] - time if prompt == 0 else end_timestamp[i] - time
          time_spent += time
    data[i].append(time_spent / timedelta(minutes=30))
  return data

#### Initial Prompt -> Middle Prompt

In [755]:
columns.append("validated_time_rate_1")

In [756]:
data = validated_time_rate(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

In [757]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1
191,0.840556
225,0.000000
234,0.514444
240,0.296111
242,0.465556


#### Middle Prompt -> Final Prompt

In [758]:
columns.append("validated_time_rate_2")

In [759]:
data = validated_time_rate(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

In [760]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2
191,0.840556,0.010000
225,0.000000,0.299444
234,0.514444,0.121667
240,0.296111,0.149444
242,0.465556,0.000000


### Mean of Highest version validated

Indicator name : 
* highest_version_mean_validated

Computation : 
* x = [for each subject validated : max version validated]
* sum(x) / count(x)

In [761]:
def highest_version_mean_validated(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  column_val_sujet = list(df["validation"].columns).index("id_sujet")
  column_val_version = list(df["validation"].columns).index("version")
  for i,id in enumerate(nb_connection_1h):
    df_val = df["validation"].query("id_connexion == {} and score > 0 and experimentation == 0 and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    versions = defaultdict(int)
    for j in range(df_val.shape[0]):
      if versions[df_val.iloc[j,column_val_sujet]] < df_val.iloc[j,column_val_version]:
        versions[df_val.iloc[j,column_val_sujet]] = df_val.iloc[j,column_val_version]
    
    data[i].append(sum([versions[key] for key in versions.keys()]) / len(versions) if len(versions) != 0 else 0)
  return data

#### Initial Prompt -> Middle Prompt

In [762]:
columns.append("highest_version_mean_validated_1")

In [763]:
data = highest_version_mean_validated(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

In [764]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1
191,0.840556,0.010000,3.5
225,0.000000,0.299444,0.0
234,0.514444,0.121667,2.4
240,0.296111,0.149444,4.0
242,0.465556,0.000000,3.0


#### Middle Prompt -> Final Prompt

In [765]:
columns.append("highest_version_mean_validated_2")

In [766]:
data = highest_version_mean_validated(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

In [767]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2
191,0.840556,0.010000,3.5,3.0
225,0.000000,0.299444,0.0,3.0
234,0.514444,0.121667,2.4,3.0
240,0.296111,0.149444,4.0,3.0
242,0.465556,0.000000,3.0,0.0


### Time spent on Help module

Indicator name : 
* time_consulting_help

Computation : 
* sum([For each subject : time spent on the help module])

In [768]:
def time_consulting_help(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  column_nav_sujet = list(df["navigation"].columns).index("id_sujet")
  column_nav_version = list(df["navigation"].columns).index("version")
  column_nav_module = list(df["navigation"].columns).index("module")
  column_nav_timestamp = list(df["navigation"].columns).index("timestamp")
  for i,id in enumerate(nb_connection_1h):
    df_nav = df["navigation"].query("id_connexion == {} and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_nav, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_nav, middle_timestamp[i], end_timestamp[i])
    df_nav = df_nav.iloc[index,:]
    time_spent = timedelta(seconds=0)
    for j in range(df_nav.shape[0]):
      if df_nav.iloc[j,column_nav_module] == 'Aide':
        sujet = df_nav.iloc[j,column_nav_sujet]
        version = df_nav.iloc[j,column_nav_version]
        time = df_nav.iloc[j,column_nav_timestamp]
        for k in range(j+1, df_nav.shape[0]):
          if df_nav.iloc[k,column_nav_sujet] != sujet or df_nav.iloc[k,column_nav_version] != version or df_nav.iloc[k,column_nav_module] != 'Aide':
            time = df_nav.iloc[k,column_nav_timestamp] - time
            break
        else:
            time = middle_timestamp[i] - time if prompt == 0 else end_timestamp[i] - time
        time_spent += time
    data[i].append(time_spent.total_seconds())
  return data

#### Initial Prompt -> Middle Prompt

In [769]:
columns.append("time_consulting_help_1")

In [770]:
data = time_consulting_help(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

In [771]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1
191,0.840556,0.010000,3.5,3.0,0.0
225,0.000000,0.299444,0.0,3.0,73.0
234,0.514444,0.121667,2.4,3.0,62.0
240,0.296111,0.149444,4.0,3.0,10.0
242,0.465556,0.000000,3.0,0.0,0.0


#### Middle Prompt -> Final Prompt

In [772]:
columns.append("time_consulting_help_2")

In [773]:
data = time_consulting_help(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

In [774]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0
225,0.000000,0.299444,0.0,3.0,73.0,2.0
234,0.514444,0.121667,2.4,3.0,62.0,0.0
240,0.296111,0.149444,4.0,3.0,10.0,13.0
242,0.465556,0.000000,3.0,0.0,0.0,4.0


### Mean of experimentation+Validation number / subject experimented

Indicator name : 
* mean_test_per_subject

Computation : 
* x = [experimentation number per subject experimented]
* sum(x) / count(x)

In [775]:
def mean_test_per_subject(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  for i,id in enumerate(nb_connection_1h):
    df_val = df["validation"].query("id_connexion == {} and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    exp_number = list(df_val[["id_sujet", "version"]].value_counts())
    data[i].append(sum(exp_number) / len(exp_number) if len(exp_number) != 0 else 0)
  return data

#### Initial Prompt -> Middle Prompt

In [776]:
columns.append("mean_test_per_subject_1")

In [777]:
data = mean_test_per_subject(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

In [778]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000


#### Middle Prompt -> Final Prompt

In [779]:
columns.append("mean_test_per_subject_2")

In [780]:
data = mean_test_per_subject(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

In [781]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000


### Mean Size Modification per test

Indicator name : 
* mean_size_modification_per_test

Computation : 
* x = [modification number per subject experimented]
* sum(x) / count(x)

In [782]:
def mean_size_modification_per_test(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  column_val_sujet = list(df["validation"].columns).index("id_sujet")
  column_val_version = list(df["validation"].columns).index("version")
  column_val_timestamp = list(df["validation"].columns).index("timestamp")
  column_mod_size = list(df["modification"].columns).index("taille_reponse")
  for i,id in enumerate(nb_connection_1h):
    df_val = df["validation"].query("id_connexion == {} and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    modifications_size = []
    for j in range(df_val.shape[0]):
      sujet = df_val.iloc[j, column_val_sujet]
      version = df_val.iloc[j, column_val_version]
      timestamp = df_val.iloc[j, column_val_timestamp]
      column_val_timestamp
      df_mod = df["modification"].query("id_connexion == {} and id_sujet == {} and version == {} and id_sujet in {}".format(id, sujet, version, sujet_range))
      index = get_index_timestamp(df_mod, beginning_timestamp[i], timestamp)
      df_mod = df_mod.iloc[index,:]
      modifications_size.append(df_mod.iloc[df_mod.shape[0]-1, column_mod_size] if df_mod.shape[0] != 0 else 0)
    if len(modifications_size) != 0:
      for j in range(len(modifications_size)-1,0,-1):
        modifications_size[j] = abs(modifications_size[j]-modifications_size[j-1])
    data[i].append(sum(modifications_size) / len(modifications_size) if len(modifications_size) != 0 else 0)
  return data

#### Initial Prompt -> Middle Prompt

In [783]:
columns.append("mean_size_modification_per_test_1")

In [784]:
data = mean_size_modification_per_test(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

In [785]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000


#### Middle Prompt -> Final Prompt

In [786]:
columns.append("mean_size_modification_per_test_2")

In [787]:
data = mean_size_modification_per_test(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

In [788]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444


### Mean of modification per subject experimented or validated

Indicator name : 
* mean_modification_per_subject

Computation : 
* x = [modification number per subject experimented]
* sum(x) / count(x)

In [789]:
def mean_modification_per_subject(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  column_mod_sujet = list(df["modification"].columns).index("id_sujet")
  column_mod_version = list(df["modification"].columns).index("version")
  column_mod_timestamp = list(df["modification"].columns).index("timestamp")
  for i,id in enumerate(nb_connection_1h):
    df_val = df["validation"].query("id_connexion == {} and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    comb = list(df_val[["id_sujet", "version"]].value_counts().index)
    modifications = []
    for s,v in comb:
      df_mod = df["modification"].query("id_connexion == {} and id_sujet == {} and version == {} and id_sujet in {}".format(id,s,v, sujet_range))
      index = get_index_timestamp(df_mod, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_mod, middle_timestamp[i], end_timestamp[i])
      df_mod = df_mod.iloc[index,:]
      index = eliminate_same_subject_version_other_following(df_mod, column_mod_sujet, column_mod_version, column_mod_timestamp)
      df_mod = df_mod.iloc[index,:]
      modifications.append(df_mod.shape[0])
    data[i].append(sum(modifications) / len(modifications) if len(modifications) != 0 else 0)
  return data

#### Initial Prompt -> Middle Prompt

In [790]:
columns.append("mean_modification_per_subject_1")

In [791]:
data = mean_modification_per_subject(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

In [792]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00


#### Middle Prompt -> Final Prompt

In [793]:
columns.append("mean_modification_per_subject_2")

In [794]:
data = mean_modification_per_subject(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

In [795]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000


### Mean Time Active

Indicator name : 
* mean_time_active

Computation : 
* Mean time between each (next action - previous action)

In [796]:
def mean_time_active(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, tables, sujet_range, prompt=0):
  for i in tqdm.tqdm(range(len(nb_connection_1h)), position=0, leave=True):
    id = nb_connection_1h[i]
    own_df = {}
    index = {}
    columns_memory = {}
    for table in tables:
      df_table = df[table].query("id_connexion == {} and id_sujet in {}".format(id, sujet_range))
      indexes = get_index_timestamp(df_table, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_table, middle_timestamp[i], end_timestamp[i])
      own_df[table] = df_table.iloc[indexes,:]
      index[table] = 0
      columns_memory[table] = list(df_table.columns).index("timestamp")
    # table souris (too low if we filter it with pandas due to the size of the table)
    mysql_connection = get_connection(_db_config)
    with mysql_connection.cursor() as cursor:
      try:
        # ----------- Get every connection / participant ---------- #
        query_select = "SELECT * FROM souris WHERE id_connexion = %s ORDER BY timestamp"
        cursor.execute(query_select, (id,))
        result = cursor.fetchall()
        columns_name = np.array([cursor.description[i][0] for i in range(len(cursor.description)) if cursor.description[i][0]])
        tempdata = []
        for row in result:
          tempdata.append(np.array([row[i] for i in range(len(row))]))
        own_df["souris"] = pd.DataFrame(tempdata, columns=columns_name)
        indexes = get_index_timestamp(own_df["souris"], beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(own_df["souris"], middle_timestamp[i], end_timestamp[i])
        own_df["souris"] = own_df["souris"].iloc[indexes,:]
        index["souris"] = 0
        columns_memory["souris"] = list(own_df["souris"].columns).index("timestamp")
      except Exception:
        traceback.print_exc()
    all_seen = False
    count_time = timedelta(0)
    count = 0
    memory_timestamp = None
    while not all_seen:
      timestamps = {}
      for table in tables+["souris"]:
        if index[table] <= own_df[table].shape[0]-1:
          timestamps[table] = own_df[table].iloc[index[table], columns_memory[table]]
      keys = [key for key in timestamps.keys()]
      if len(keys) == 0:
        if memory_timestamp != None:
          count_time += (middle_timestamp[i] if prompt == 0 else end_timestamp[i]) - memory_timestamp
        count += 1
        all_seen = True
      else:
        key = keys[np.argmin([timestamps[k] for k in timestamps.keys()])]
        if memory_timestamp == None:
          memory_timestamp = timestamps[key]
          index[key] += 1
        else:
          count_time += timestamps[key] - memory_timestamp
          count += 1
          memory_timestamp = timestamps[key]
          index[key] += 1
    data[i].append((count_time / count if count != 0 else 0).total_seconds())
  return data

#### Initial Prompt -> Middle Prompt

In [797]:
columns.append("mean_time_active_1")

In [798]:
data = mean_time_active(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, _tables[:4] + _tables[7:8], sujet_range, prompt=0)

100%|██████████| 89/89 [05:06<00:00,  3.44s/it]


In [799]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000,0.526670
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000,0.860975
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000,0.610504
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333,1.533905
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000,4.162162


#### Middle Prompt -> Final Prompt

In [800]:
columns.append("mean_time_active_2")

In [801]:
data = mean_time_active(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, _tables[:4] + _tables[7:8], sujet_range, prompt=1)

100%|██████████| 89/89 [04:22<00:00,  2.95s/it]


In [802]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1,mean_time_active_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000,0.526670,0.512549
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000,0.860975,0.834028
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000,0.610504,1.958195
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333,1.533905,1.501677
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000,4.162162,0.630589


### Quality of Experimentation

Indicator name : 
* experimentation_quality

Computation : 
* x = has consulted help before (for the same subject)
* y = min(max(modification before last experimentation,10),10)/10
* z = has last modification not an error
* a = [for each experimentation : 0.25x + 0.25y + 0.5z]
* sum(a)/count(a)

In [803]:
def experimentation_quality(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  column_val_sujet = list(df["validation"].columns).index("id_sujet")
  column_val_version = list(df["validation"].columns).index("version")
  column_val_timestamp = list(df["validation"].columns).index("timestamp")
  column_nav_sujet = list(df["navigation"].columns).index("id_sujet")
  column_nav_version = list(df["navigation"].columns).index("version")
  column_nav_timestamp = list(df["navigation"].columns).index("timestamp")
  column_mod_erreur = list(df["modification"].columns).index("erreur")
  CONST_MOD_NUMBER = 10

  for i in tqdm.tqdm(range(len(nb_connection_1h)), position=0, leave=True):
    id = nb_connection_1h[i]
    has_consulted = []
    mod_number = []
    has_error = []

    df_val = df["validation"].query("id_connexion == {} and experimentation == 1 and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    for j in range(df_val.shape[0]):
      sujet = df_val.iloc[j, column_val_sujet]
      version = df_val.iloc[j, column_val_version]
      timestamp = df_val.iloc[j, column_val_timestamp]

      # Has consulted
      df_nav = df["navigation"].query("id_connexion == {}".format(id))
      df_nav_aide = df["navigation"].query("id_connexion == {} and id_sujet == {} and version == {} and module == 'Aide' and id_sujet in {}".format(id, sujet, version, sujet_range))
      index = get_index_timestamp(df_nav, beginning_timestamp[i], timestamp) if prompt == 0 else get_index_timestamp(df_nav, middle_timestamp[i], timestamp)
      df_nav = df_nav.iloc[index,:]
      index = get_index_timestamp(df_nav_aide, beginning_timestamp[i], timestamp) if prompt == 0 else get_index_timestamp(df_nav_aide, middle_timestamp[i], timestamp)
      df_nav_aide = df_nav_aide.iloc[index,:]
      if df_nav_aide.shape[0] != 0:
        aide_timestamp = df_nav_aide.iloc[df_nav_aide.shape[0] - 1, column_nav_timestamp]
        for k in range(df_nav.shape[0]-1, -1, -1):
          if df_nav.iloc[k, column_nav_sujet] == sujet and df_nav.iloc[k, column_nav_version] == version:
            if df_nav.iloc[k, column_nav_timestamp] < aide_timestamp:
              has_consulted.append(1)
              break
            else:
              continue
          else:
            has_consulted.append(0)
            break
      else:
        has_consulted.append(0)
      
      # modification number
      if j == 0:
        min_timestamp = beginning_timestamp[i] if timestamp == 0 else middle_timestamp[i]
      else:
        min_timestamp = df_val.iloc[j-1, column_val_timestamp]
        if prompt == 0 and min_timestamp < middle_timestamp[i]:
          min_timestamp = middle_timestamp[i]
      df_mod = df["modification"].query("id_connexion == {} and id_sujet == {} and version == {} and id_sujet in {}".format(id, sujet, version, sujet_range))
      index = get_index_timestamp(df_mod, min_timestamp, timestamp)
      df_mod = df_mod.iloc[index,:]
      mod_number.append(min(max(df_mod.shape[0],CONST_MOD_NUMBER),CONST_MOD_NUMBER)/CONST_MOD_NUMBER)

      # has last modification an error
      if df_mod.shape[0] == 0:
        has_error.append(0)
      else:
        has_error.append(1 if df_mod.iloc[df_mod.shape[0]-1, column_mod_erreur] == 'Aucune' else 0)

    experimentation_qualities = [has_consulted[expe]*0.25 + mod_number[expe]*0.25 + has_error[expe]*0.5 for expe in range(len(has_consulted))]
    data[i].append(sum(experimentation_qualities) / len(experimentation_qualities) if len(experimentation_qualities) != 0 else 0)
  return data

#### Initial Prompt -> Middle Prompt

In [804]:
columns.append("experimentation_quality_1")

In [805]:
data = experimentation_quality(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

100%|██████████| 89/89 [00:23<00:00,  3.78it/s]


In [806]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1,mean_time_active_2,experimentation_quality_1
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000,0.526670,0.512549,0.000000
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000,0.860975,0.834028,0.302083
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000,0.610504,1.958195,0.250000
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333,1.533905,1.501677,0.281250
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000,4.162162,0.630589,0.000000


#### Middle Prompt -> Final Prompt

In [807]:
columns.append("experimentation_quality_2")

In [808]:
data = experimentation_quality(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

100%|██████████| 89/89 [00:10<00:00,  8.52it/s]


In [809]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1,mean_time_active_2,experimentation_quality_1,experimentation_quality_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000,0.526670,0.512549,0.000000,0.75
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000,0.860975,0.834028,0.302083,0.75
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000,0.610504,1.958195,0.250000,0.75
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333,1.533905,1.501677,0.281250,1.00
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000,4.162162,0.630589,0.000000,0.00


### Quality of validation

Indicator name : 
* validation_quality

Computation : 
* x = Has at least 1 minute on the subject before validating
* y = Has at least 1 minute before re-validating
* z = Has not an error
* a = Has done at least one experimentation before validating
* b = [for each validation : 0.1x + 0.15y + 0.5z + 0.25a]
* sum(b)/count(b)

In [810]:
def validation_quality(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  column_val_sujet = list(df["validation"].columns).index("id_sujet")
  column_val_version = list(df["validation"].columns).index("version")
  column_val_timestamp = list(df["validation"].columns).index("timestamp")
  column_val_erreur = list(df["validation"].columns).index("type_erreur")
  column_nav_sujet = list(df["navigation"].columns).index("id_sujet")
  column_nav_version = list(df["navigation"].columns).index("version")
  column_nav_timestamp = list(df["navigation"].columns).index("timestamp")
  CONST_MINUTE = timedelta(minutes=1)

  for i in tqdm.tqdm(range(len(nb_connection_1h)), position=0, leave=True):
    id = nb_connection_1h[i]
    time_before_validation = []
    time_before_revalidation = []
    one_exp_before = []
    has_error = []

    df_val = df["validation"].query("id_connexion == {} and experimentation == 0 and id_sujet in {}".format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    for j in range(df_val.shape[0]):
      sujet = df_val.iloc[j, column_val_sujet]
      version = df_val.iloc[j, column_val_version]
      timestamp = df_val.iloc[j, column_val_timestamp]

      # has an error
      has_error.append(1 if df_val.iloc[j, column_val_erreur] == 'Aucune' else 0)

      # Has done at least one experimentation before
      if j == 0:
        min_timestamp = beginning_timestamp[i] if prompt == 0 else middle_timestamp[i]
      else:
        min_timestamp = df_val.iloc[j-1, column_val_timestamp]
        if prompt == 0 and min_timestamp < middle_timestamp[i]:
          min_timestamp = middle_timestamp[i]
      df_val_exp = df["validation"].query("id_connexion == {} and experimentation == 1 and id_sujet == {} and version == {} and id_sujet in {}".format(id, sujet, version, sujet_range))
      index = get_index_timestamp(df_val_exp, min_timestamp, timestamp)
      df_val_exp = df_val_exp.iloc[index,:]
      one_exp_before.append(1 if df_val_exp.shape[0] > 0 else 0)
      
      # time before validation
      df_nav = df["navigation"].query("id_connexion == {} and id_sujet == {} and version == {} and id_sujet in {}".format(id, sujet, version, sujet_range))
      index = get_index_timestamp(df_nav, beginning_timestamp[i], timestamp) if prompt == 0 else get_index_timestamp(df_nav, middle_timestamp[i], timestamp)
      df_nav = df_nav.iloc[index,:]
      if df_nav.shape[0] == 0:
        min_timestamp = beginning_timestamp[i] if prompt == 0 else middle_timestamp[i]
        time_before_validation.append(min_timestamp + CONST_MINUTE <= timestamp)
      else:
        time_before_validation.append(1 if df_nav.iloc[df_nav.shape[0] - 1, column_nav_timestamp] + CONST_MINUTE <= timestamp else 0)

      # time before re-validation
      if j == 0:
        time_before_revalidation.append(time_before_validation[-1])
      else:
        time_before_revalidation.append(1 if df_val.iloc[j-1, column_val_timestamp] + CONST_MINUTE <= timestamp else 0)

    validation_qualities = [time_before_validation[val]*0.1 + time_before_revalidation[val]*0.15 + one_exp_before[val]*0.25 + has_error[val]*0.5 for val in range(len(has_error))]
    data[i].append(sum(validation_qualities) / len(validation_qualities) if len(validation_qualities) != 0 else 0)
  return data

#### Initial Prompt -> Middle Prompt

In [811]:
columns.append("validation_quality_1")

In [812]:
data = validation_quality(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)

100%|██████████| 89/89 [00:19<00:00,  4.57it/s]


In [813]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1,mean_time_active_2,experimentation_quality_1,experimentation_quality_2,validation_quality_1
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000,0.526670,0.512549,0.000000,0.75,0.276923
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000,0.860975,0.834028,0.302083,0.75,0.000000
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000,0.610504,1.958195,0.250000,0.75,0.256000
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333,1.533905,1.501677,0.281250,1.00,0.150926
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000,4.162162,0.630589,0.000000,0.00,0.500000


#### Middle Prompt -> Final Prompt

In [814]:
columns.append("validation_quality_2")

In [815]:
data = validation_quality(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

100%|██████████| 89/89 [00:15<00:00,  5.68it/s]


In [816]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1,mean_time_active_2,experimentation_quality_1,experimentation_quality_2,validation_quality_1,validation_quality_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000,0.526670,0.512549,0.000000,0.75,0.276923,0.168333
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000,0.860975,0.834028,0.302083,0.75,0.000000,0.520000
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000,0.610504,1.958195,0.250000,0.75,0.256000,0.316667
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333,1.533905,1.501677,0.281250,1.00,0.150926,0.134314
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000,4.162162,0.630589,0.000000,0.00,0.500000,0.105556


### Score

In [817]:
def get_cumuled_score(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0):
  cumul_score = []
  for i,id in enumerate(nb_connection_1h):
    scores = []
    df_val = df["validation"].query('id_connexion == {} and experimentation == 0 and id_sujet in {}'.format(id, sujet_range))
    index = get_index_timestamp(df_val, beginning_timestamp[i], middle_timestamp[i]) if prompt == 0 else get_index_timestamp(df_val, middle_timestamp[i], end_timestamp[i])
    df_val = df_val.iloc[index,:]
    for id_sujet in list(set(df_val.id_sujet)):
      val_version = list(df_val.query('id_sujet == {} and score > 0'.format(id_sujet)).version)
      versions = np.zeros((5,))
      for row in range(len(val_version)):
        versions[val_version[row]] = val_version[row] * 10
      scores.append(np.max(versions))
    data[i].append(np.sum(scores))
  return data

In [818]:
columns.append("score_1")
columns.append("score_2")

In [819]:
data = get_cumuled_score(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=0)
data = get_cumuled_score(df, nb_connection_1h, data, beginning_timestamp, middle_timestamp, end_timestamp, sujet_range, prompt=1)

In [820]:
draw(data, columns, nb_connection_1h)

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1,mean_time_active_2,experimentation_quality_1,experimentation_quality_2,validation_quality_1,validation_quality_2,score_1,score_2
191,0.840556,0.010000,3.5,3.0,0.0,0.0,3.250,15.500000,296.230769,291.774194,71.75,91.000000,0.526670,0.512549,0.000000,0.75,0.276923,0.168333,70.0,30.0
225,0.000000,0.299444,0.0,3.0,73.0,2.0,4.000,4.000000,164.541667,233.000000,24.50,27.800000,0.860975,0.834028,0.302083,0.75,0.000000,0.520000,0.0,60.0
234,0.514444,0.121667,2.4,3.0,62.0,0.0,4.125,2.500000,226.939394,926.400000,25.75,31.000000,0.610504,1.958195,0.250000,0.75,0.256000,0.316667,120.0,30.0
240,0.296111,0.149444,4.0,3.0,10.0,13.0,15.500,8.666667,105.500000,131.692308,88.00,52.833333,1.533905,1.501677,0.281250,1.00,0.150926,0.134314,40.0,30.0
242,0.465556,0.000000,3.0,0.0,0.0,4.0,1.000,27.000000,1788.000000,87.444444,2.00,169.000000,4.162162,0.630589,0.000000,0.00,0.500000,0.105556,30.0,0.0


## Rearrange Brut Dataset

### Dataset

#### Data Save

In [821]:
dataset_brut = pd.DataFrame(data, columns=columns, index=nb_connection_1h)

In [822]:
dataset_brut.to_csv("dataset_brut_dbnv2.csv")

#### Data Loading

In [823]:
dataset_brut = pd.read_csv("dataset_brut_dbnv2.csv", index_col=0, header=0)

In [824]:
dataset_brut

,validated_time_rate_1,validated_time_rate_2,highest_version_mean_validated_1,highest_version_mean_validated_2,time_consulting_help_1,time_consulting_help_2,mean_test_per_subject_1,mean_test_per_subject_2,mean_size_modification_per_test_1,mean_size_modification_per_test_2,mean_modification_per_subject_1,mean_modification_per_subject_2,mean_time_active_1,mean_time_active_2,experimentation_quality_1,experimentation_quality_2,validation_quality_1,validation_quality_2,score_1,score_2
191,0.840556,0.010000,3.500000,3.000000,0.0,0.0,3.250000,15.500000,296.230769,291.774194,71.750000,91.000000,0.526670,0.512549,0.000000,0.7500,0.276923,0.168333,70.0,30.0
225,0.000000,0.299444,0.000000,3.000000,73.0,2.0,4.000000,4.000000,164.541667,233.000000,24.500000,27.800000,0.860975,0.834028,0.302083,0.7500,0.000000,0.520000,0.0,60.0
234,0.514444,0.121667,2.400000,3.000000,62.0,0.0,4.125000,2.500000,226.939394,926.400000,25.750000,31.000000,0.610504,1.958195,0.250000,0.7500,0.256000,0.316667,120.0,30.0
240,0.296111,0.149444,4.000000,3.000000,10.0,13.0,15.500000,8.666667,105.500000,131.692308,88.000000,52.833333,1.533905,1.501677,0.281250,1.0000,0.150926,0.134314,40.0,30.0
242,0.465556,0.000000,3.000000,0.000000,0.0,4.0,1.000000,27.000000,1788.000000,87.444444,2.000000,169.000000,4.162162,0.630589,0.000000,0.0000,0.500000,0.105556,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5208,0.596111,0.231111,4.000000,3.000000,0.0,0.0,17.000000,8.000000,129.088235,221.125000,99.500000,84.500000,0.462565,0.432217,0.250000,0.7500,0.152083,0.200000,40.0,30.0
5209,0.000000,0.250000,0.000000,4.000000,4.0,0.0,2.000000,5.333333,394.333333,314.562500,44.666667,49.666667,0.403295,0.434436,0.250000,0.5625,0.000000,0.171875,0.0,80.0
5210,0.341111,0.148333,3.000000,2.000000,68.0,57.0,7.600000,5.400000,200.631579,266.111111,49.800000,58.400000,0.480954,0.520694,0.250000,0.0000,0.163889,0.153704,60.0,20.0
5212,0.616111,0.259444,2.666667,3.333333,0.0,0.0,8.800000,5.250000,72.068182,550.000000,46.400000,73.750000,0.447123,0.368647,0.250000,0.7500,0.184375,0.250000,80.0,100.0


In [875]:
def replace_by_classes_dbn(dataset_brut, columns):
  columns["validated_time_rate_1"] = list(pd.cut(dataset_brut.validated_time_rate_1, bins=[-1.,0,0.2,0.4,0.6,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"]))
  columns["validated_time_rate_2"] = list(pd.cut(dataset_brut.validated_time_rate_2, bins=[-1.,0,0.2,0.4,0.6,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"]))
  columns["highest_version_mean_validated_1"] = pd.cut(dataset_brut.highest_version_mean_validated_1, bins=[-1,1,2,3,4], labels=["Classe1","Classe2","Classe3","Classe4"])
  columns["highest_version_mean_validated_2"] = pd.cut(dataset_brut.highest_version_mean_validated_2, bins=[-1,1,2,3,4], labels=["Classe1","Classe2","Classe3","Classe4"])
  columns["time_consulting_help_1"] = pd.cut(dataset_brut.time_consulting_help_1, bins=[-1,0,10,30,120,10000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["time_consulting_help_2"] = pd.cut(dataset_brut.time_consulting_help_2, bins=[-1,0,10,30,120,10000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["mean_test_per_subject_1"] = pd.cut(dataset_brut.mean_test_per_subject_1, bins=[-1,2,5,8,11,15,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5","Classe6"])
  columns["mean_test_per_subject_2"] = pd.cut(dataset_brut.mean_test_per_subject_2, bins=[-1,2,5,8,11,15,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5","Classe6"])
  #columns["mean_size_modification_per_test_1"] = pd.cut(dataset_brut.mean_size_modification_per_test_1, bins=[-1,100,200,300,400,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  #columns["mean_size_modification_per_test_2"] = pd.cut(dataset_brut.mean_size_modification_per_test_2, bins=[-1,100,200,300,400,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["mean_modification_per_subject_1"] = pd.cut(dataset_brut.mean_modification_per_subject_1, bins=[-1,0,20,50,100,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["mean_modification_per_subject_2"] = pd.cut(dataset_brut.mean_modification_per_subject_2, bins=[-1,0,20,50,100,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["mean_time_active_1"] = pd.cut(dataset_brut.mean_time_active_1, bins=[-1,0,0.3,0.6,1,5,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5","Classe6"])
  columns["mean_time_active_2"] = pd.cut(dataset_brut.mean_time_active_2, bins=[-1,0,0.3,0.6,1,5,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5","Classe6"])
  columns["experimentation_quality_1"] = pd.cut(dataset_brut.experimentation_quality_1, bins=[-1,0,0.2,0.4,0.5,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["experimentation_quality_2"] = pd.cut(dataset_brut.experimentation_quality_2, bins=[-1,0,0.2,0.4,0.5,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["validation_quality_1"] = pd.cut(dataset_brut.validation_quality_1, bins=[-1,0,0.1,0.2,0.4,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["validation_quality_2"] = pd.cut(dataset_brut.validation_quality_2, bins=[-1,0,0.1,0.2,0.4,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])

  #columns["score_1"] = pd.cut(dataset_brut.score_1, bins=[-1,99,400], labels=["Low","High"])
  #columns["score_2"] = pd.cut(dataset_brut.score_2, bins=[-1,99,400], labels=["Low","High"])
  columns["score_1"] = pd.cut(dataset_brut.score_1, bins=[-1,np.percentile(dataset_brut.score_1, 33),np.percentile(dataset_brut.score_1, 66),np.percentile(dataset_brut.score_1, 100)], labels=["Low","Medium","High"])
  columns["score_2"] = pd.cut(dataset_brut.score_2, bins=[-1,np.percentile(dataset_brut.score_2, 33),np.percentile(dataset_brut.score_2, 66),np.percentile(dataset_brut.score_2, 100)], labels=["Low","Medium","High"])
  return columns

In [860]:
def replace_by_classes_bn(dataset_brut, columns):
  columns["validated_time_rate"] = list(pd.cut(dataset_brut.validated_time_rate, bins=[-1.,0,0.2,0.4,0.6,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"]))
  columns["highest_version_mean_validated"] = pd.cut(dataset_brut.highest_version_mean_validated, bins=[-1,1,2,3,4], labels=["Classe1","Classe2","Classe3","Classe4"])
  columns["time_consulting_help"] = pd.cut(dataset_brut.time_consulting_help, bins=[-1,0,10,30,120,10000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["mean_test_per_subject"] = pd.cut(dataset_brut.mean_test_per_subject, bins=[-1,2,5,8,11,15,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5","Classe6"])
  #columns["mean_size_modification_per_test"] = pd.cut(dataset_brut.mean_size_modification_per_test, bins=[-1,100,200,300,400,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["mean_modification_per_subject"] = pd.cut(dataset_brut.mean_modification_per_subject, bins=[-1,20,30,40,50,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["mean_time_active"] = pd.cut(dataset_brut.mean_time_active, bins=[-1,0,0.3,0.6,1,5,50000], labels=["Classe1","Classe2","Classe3","Classe4","Classe5","Classe6"])
  columns["experimentation_quality"] = pd.cut(dataset_brut.experimentation_quality, bins=[-1,0,0.2,0.4,0.5,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  columns["validation_quality"] = pd.cut(dataset_brut.validation_quality, bins=[-1,0,0.1,0.2,0.4,1], labels=["Classe1","Classe2","Classe3","Classe4","Classe5"])
  # columns["score"] = pd.cut(dataset_brut.score, bins=[-1,99,400], labels=["Low","High"])
  columns["score"] = pd.cut(dataset_brut.score, bins=[-1,np.percentile(dataset_brut.score, 33),np.percentile(dataset_brut.score, 66),np.percentile(dataset_brut.score, 100)], labels=["Low","Medium","High"])
  return columns

### DBN

In [876]:
data_columns = replace_by_classes_dbn(dataset_brut, {})

In [877]:
final_dataset = pd.DataFrame(data_columns, index=nb_connection_1h)

In [878]:
final_dataset.to_csv("dbn_v2.csv", index=False)

### To bayesian Network

In [861]:
bn_data_columns = {}
keys = ["validated_time_rate", "highest_version_mean_validated", "time_consulting_help", "mean_test_per_subject","mean_size_modification_per_test", "mean_modification_per_subject", 
        "mean_time_active", "experimentation_quality", "validation_quality", "score"]
for i in keys:
  new_col = []
  one = list(dataset_brut[i+"_1"])
  two = list(dataset_brut[i+"_2"])
  for j in range(len(one)):
    if i != "score":
      new_col.append((one[j] + two[j]) / 2)
    else:
      new_col.append(one[j] + two[j])
  bn_data_columns[i] = new_col

In [862]:
final_dataset_bn = pd.DataFrame(bn_data_columns, index=nb_connection_1h)

In [863]:
bn_data_columns_final = replace_by_classes_bn(final_dataset_bn, {})

In [864]:
final_dataset_bn = pd.DataFrame(bn_data_columns_final, index=nb_connection_1h)
final_dataset_bn

,validated_time_rate,highest_version_mean_validated,time_consulting_help,mean_test_per_subject,mean_modification_per_subject,mean_time_active,experimentation_quality,validation_quality,score
191,Classe4,Classe4,Classe1,Classe4,Classe5,Classe3,Classe3,Classe4,Medium
225,Classe2,Classe2,Classe4,Classe2,Classe2,Classe4,Classe5,Classe4,Low
234,Classe3,Classe3,Classe4,Classe2,Classe2,Classe5,Classe4,Classe4,High
240,Classe3,Classe4,Classe3,Classe5,Classe5,Classe5,Classe5,Classe3,Medium
242,Classe3,Classe2,Classe2,Classe5,Classe5,Classe5,Classe1,Classe4,Low
...,...,...,...,...,...,...,...,...,...
5208,Classe4,Classe4,Classe1,Classe5,Classe5,Classe3,Classe4,Classe3,Medium
5209,Classe2,Classe2,Classe2,Classe2,Classe4,Classe3,Classe4,Classe2,Medium
5210,Classe3,Classe3,Classe4,Classe3,Classe5,Classe3,Classe2,Classe3,Medium
5212,Classe4,Classe3,Classe1,Classe3,Classe5,Classe3,Classe4,Classe4,High


In [865]:
final_dataset_bn.to_csv("bn_v2.csv", index=False)

In [866]:
a = np.sort(np.array(bn_data_columns["score"]), axis=0)
print(a, np.percentile(a,33), np.percentile(a,66))
400/3

[  0.   0.   0.   0.   0.  20.  20.  20.  20.  20.  30.  30.  30.  30.
  40.  40.  40.  40.  40.  40.  50.  50.  50.  50.  50.  50.  60.  60.
  60.  60.  60.  60.  60.  60.  60.  60.  60.  60.  60.  70.  70.  70.
  70.  70.  70.  70.  70.  80.  80.  80.  80.  80.  80.  90.  90.  90.
  90. 100. 100. 100. 100. 100. 100. 110. 110. 120. 130. 130. 130. 130.
 140. 140. 140. 150. 150. 160. 160. 170. 180. 180. 180. 180. 180. 190.
 200. 200. 210. 250. 320.] 60.0 100.0


133.33333333333334